In [90]:
import spacy
from spacy.matcher import Matcher
from IPython.display import Markdown, display
import os
import PyPDF2
import re
from IPython.display import clear_output
import xml.etree.ElementTree as ET
from spacy import displacy
from itertools import combinations

In [2]:
spacy.prefer_gpu()
nlp = spacy.load("nl_core_news_lg")

base = r'F:\Data files\Master thesis\\'

# displacy.render(doc,style="ent",jupyter=True)

In [3]:
def printHilight(string):
    print('\x1b[1;31m'+string + ' ' +'\x1b[0m', end='')

In [4]:
# extract text from pdfs in directory base
def extractText(base):
    
    # make dir to store extracted text
    if not os.path.isdir(base + 'text'):
        os.mkdir(base + 'text')
    
    # process all pdfs in base dir
    for item in os.listdir(base):
        allText = ''
        
        # skip directories
        if os.path.isdir(base + item):
            continue

        # try to extract all text from pdf
        try:
            with open(base + item, 'rb') as f:
                reader = PyPDF2.PdfFileReader(f)
                
                # save text from every page and clean it a bit
                for page in reader.pages:
                    text = page.extractText()
                    text = text.replace('\n', ' ')
                    
                    # add page text to all text
                    allText = allText + text + '\n'

        except:
            continue
        
        # save only to text if text has more than 10 words
        if len([x for x in allText.replace('\n', '').split(' ') if x]) > 10:
            with open("..\\data\\covid wob text without ocr\\" + item + '.txt' , 'w', encoding="utf-8") as f:
                f.write(allText)

In [5]:
# this function visually prints text with named entities highlighted
# will also help with labeling data as the indexes are printed above the words
def labelData():
    
    # process all files in given dir
    for file in os.listdir('..\\data\\covid wob text without ocr'):
        
        # open and save text
        with open('..\\data\\covid wob text without ocr\\' + file, 'r', encoding='utf8') as f:
            text = f.read()

        # remove duplicated spaces and create a spacy doc opject
        text = re.sub(' +', ' ', text)
        text = text.strip()
        doc = nlp(text)
        
        # get list of entities as strings
        listTokens = [str(ent.text) for ent in doc.ents]
        
        # control variables
        nToken = 0
        tempTokens = []
        lenTemp = 0
        
        # loop over tokens
        for token in doc:
            
            # get string representation of token and add to a temporary list with token index
            t = str(token)
            tempTokens.append((t, nToken))
            
            # increase n tokens and length of all strings in tempTokens
            nToken += 1
            lenTemp += (len(t) + 1)
            
            # once lenght of all strings in tempTokens is 100 or larger, start printing the line
            if lenTemp >= 100:
                
                # print the index for every token
                for word in tempTokens:
                    
                    # if index number is more chars than the token, dont print the index number
                    if len(word[0]) < len(str(word[1])):
                        print(' ' * (len(word[0]) + 1), end='')
                        
                    # else just print index number plus a number of spaces
                    else:
                        print(word[1], ' ' * (len(word[0]) - len(str(word[1]))), end='')
                print('')
                
                # print all tokens
                for word in tempTokens:
                    
                    # highlight tokens in red if named entity
                    if word[0] in listTokens:
                        printHilight(word[0])
                    else:
                        print(word[0] + ' ', end='')
                
                # add some space between lines
                print('\n\n')
                
                # reset control variables
                tempTokens = []
                lenTemp = 0
                    
            

        return
        clear_output()
        


In [9]:
labelData()

0068ed0b40cca6270f857d2614cc63c0_besluit.pdf.txt
Korte Voorhout 7 2511 CW Den Haag Postbus 20201 2500 EE Den
0        1          2     3     4        5 6    7  8   9    10      11    12   13 14  15   16                   
Directie Juridische Zaken Korte Voorhout 7 2511 CW Den Haag Postbus 20201 2500 EE Den Haag www.rijksoverheid.nl 


17  18      19     21         22 23      25          28          29      30    31   32 33  34   35     
Ons kenmerk 2021 - 0000046938 Uw brief ( kenmerk ) > Retouradres Postbus 20201 2500 EE Den Haag Pagina 


  37    39    40 41    42   43      44      45 46 47    49      50     51       53        54      55 
1 van 3 Datum 25 maart 2021 Betreft Besluit op uw Wob - verzoek inzake agenda - afspraken Geachte In 


56 57    58  59 60   61     63        64 65 66   67     69      71  72  73     74 75 76  77           
uw brief van 10 juni 2020 , ontvangen op 15 juni 2020 , heeft u met een beroep op de Wet openbaarheid 


78  79        81       83    85       8

In [84]:
def loadTestData(path):
    
    # get contents of file
    with open(path, 'r', encoding='utf8') as f:
        text = f.read()
    
    # makes a list of strings ==> token label1 label2
    text = text.split('\n')
    
    temp = []
    
    # create list of tokens + labels
    for x in text:
        
        # try to add to list of tuples ==> (token, label1, label2)
        x = x.split(' ')
        try:
            temp.append((x[0], x[1], x[2]))
            
        # some are enters, these are represented as tuples of Nones
        except:
            temp.append((None, None, None))
    
    # create string of all text 
    text = ''
    
    # loop over tokens
    for word in temp:
        
        # none means enter
        if word[0] == None:
            text = text + '\n'
        
        # dont add extra space for punctuation
        elif word[0] in ',.!?':
            text = text + word[0]
        else:
            text = text + ' ' + word[0]
    
    # return text and list of tokens with named entity labels
    return text, temp
    
# text, labels = loadTestData(r'..\\data\\ner labeled data\\test.tsv')

# # create NLP doc object of text
# doc = nlp(text)
labels[:20]
[x[0] for x in labels if x[1] in ['B-MISC', 'I-MISC'] ]

['Belgische',
 'Europese',
 'Bosman-arrest',
 'Europese',
 'Sport',
 'Fifa-voorzitter',
 'Amerikaanse',
 'US',
 'Open',
 'Algemeen',
 'Belgische',
 'US',
 'Open',
 'Franstalige',
 'Domo-woordvoerder',
 'De',
 'Financiële',
 'Morgen',
 "LDP's",
 'Franse',
 'Algemeen',
 'De',
 'Financiële',
 'Morgen',
 'Romeinse',
 'Gothen',
 'Cultuur',
 'Titus',
 'Tony-award',
 'Disney-tekenfilm',
 'The',
 'Lion',
 'King',
 'Gothen',
 'Hans',
 'Zimmer',
 'Live',
 'in',
 'Concert',
 'Shakespeare-verfilming',
 'Titus',
 'Amerikaanse',
 '27ste',
 'Internationaal',
 'Filmfestival',
 'van',
 'Gent',
 'Gladiator',
 'The',
 'Lion',
 'King',
 'Rainman',
 'The',
 'Thin',
 'Red',
 'Line',
 'Driving',
 'Miss',
 'Daisy',
 'Peacemaker',
 'True',
 'Romance',
 'Prince',
 'of',
 'Egypt',
 'Titus',
 'Vlaams',
 'Centra',
 'voor',
 'Leerlingenbegeleiding',
 'Aula',
 'Journaal',
 'Vlaams',
 'Vlaamse',
 'Beroepssecundair',
 'Onderwijs',
 'BSO',
 'Nasdaq-index',
 'Amerikaanse',
 'De',
 'Financiële',
 'Morgen',
 'Spaanse',
 '

In [121]:
def printLabels(tempTokens):
    
    # print all index numbers
    for word in tempTokens:
        if len(word[0][0]) < len(str(word[1])):
            print(' ' * (len(word[0][0]) + 1), end='')
        else:
            print(word[1], ' ' * (len(word[0][0]) - len(str(word[1]))), end='')
    print('')
  
    for word in tempTokens:
        
        # label of 'O' means no named entity
        if word[0][1] != 'O':
            printHilight(word[0][0])
        else:
            print(word[0][0] + ' ', end='')

    print('\n')

# shows tokens with named entities highlighted
def showLabels(labels, limit = 10000):

    # set control variables
    nToken = 0
    tempTokens = []
    lenTemp = 0
    
    # only print 50 lines
    lineLimit = limit
    lines = 0
    
    # loop over tokens
    for token in labels:
        
        # if token is None, print current line
        if token[0] == None:
            lines += 1
            
            # print tokens in current line
            printLabels(tempTokens)
            
            # reset for next line
            tempTokens = []
            lenTemp = 0

        else:
            
            # add token to tempTokens and increase control variables
            tempTokens.append((token, nToken))            
            nToken += 1
            lenTemp += (len(token[0]) + 1)

            # if line length is bigger than 100, print line and reset for next line
            if lenTemp >= 100:
                lines += 1
                printLabels(tempTokens)
                tempTokens = []
                lenTemp = 0

        # stop if line limit is reached
        if lines > lineLimit:
            return
            
# showLabels(labels)

In [83]:
# get list named entities recognised by spacy
def testNER(doc):
    labelsNER = []
    
    # only get 1000 tokens
    for token in doc[:1000]:
        
        # add entities with labels
        if token.ent_type_:
            labelsNER.append((token.text, token.ent_type_))
        
        # add normal tokens with 'O' label to be consistent with test data
        else:
            labelsNER.append((token.text, 'O'))
            
    return labelsNER
        
labelsNER = testNER(doc)
# showLabels(labelsNER)

In [37]:
# test data only contains location, person and oganisation + some misc
# I tried to make the best selection of labels from spacy to reflect it
# Mostly actual things not dates and times and stuff
l = ['FAC','GPE','LANGUAGE','LOC','NORP','ORG','PERSON','PRODUCT','WORK_OF_ART']
len(doc), len(labels)

(74196, 74190)

In [114]:
def calcPreformance(doc, labels, l):
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    indexNER = 1 
    indexTest = 0
    
    falseNegativesIndexes = []
    
    for i in range(max(len(doc), len(labels))):
        if (doc[indexNER].text == labels[indexTest][0]) or (doc[indexNER].text == '\n ' and labels[indexTest][0] == None):
            tDoc = doc[indexNER]
            tLab = labels[indexTest]

            # true positive
            if tDoc.ent_type_ in l and tLab[1] != 'O':
                tp += 1

            # false positive:
            elif tDoc.ent_type_ in l and tLab[1] == 'O':
                fp += 1

            # false negative
            elif tDoc.ent_type_ not in l and tLab[1] != 'O':
                falseNegativesIndexes.append(indexTest)
                fn += 1

            # true negative
            else:
                tn += 1

        # try to sync up both indexes if they desync
        # spacy finds 6 more tokens than are in the text
        else:
            saved = False

            # check if spacy lags behind
            for j in range(100):
                if indexNER + j >= len(doc):
                    break
                
                if doc[indexNER + j].text == labels[indexTest][0]:
                    indexNER += j
                    saved = True
                    break
            if saved:
                continue
            
            # check if test lags behind
            for j in range(100):
                if indexTest + j >= len(labels):
                    indexTest
                    break
                
                if doc[indexNER].text == labels[indexTest + j][0]:
                    indexTest += j
                    saved = True
                    break
            if saved:
                continue           
            
            
#             print(f'something went wrong at indexNER {indexNER} and indexTest {indexTest}')
#             print(doc[indexNER].text == labels[indexTest][0])
#             print('|', doc[indexNER].text, '|')
#             print('|', labels[indexTest][0], '|')
#             print('')
#             print('context of spacy: ', doc[i - 10: i + 10])
#             print('context of test:', labels[i - 10:i + 10])
        
        indexNER = min(len(doc) - 1, indexNER + 1)
        indexTest = min(len(labels) - 1, indexTest + 1)
        
        if indexNER == 74195 and indexTest == 74189:
            return tp, tn, fp, fn, falseNegativesIndexes 
        
    return tp, tn, fp, fn, falseNegativesIndexes


def showResults(tp, tn, fp, fn, p):
    precision = tp / (tp + fp)
    recall = tp / (fn + tp)
    if p:
        print('True positive: ', tp)
        print('True negative: ', tn)
        print('False positive: ', fp)
        print('False negative: ', fn)
        print('Recall: ', recall)
        print('Precision: ', precision)
        print('Accuracy: ', (tp + tn) / (tp + fn + tn + fn))
        print('F1 score: ', 2 * (precision * recall) / (precision + recall))
        
    return recall, precision, (tp + tn) / (tp + fn + tn + fn), 2 * (precision * recall) / (precision + recall)


# tp, tn, fp, fn, _ = calcPreformance(doc, labels, l)
# showResults(tp, tn, fp, fn, True)


In [75]:
for x in nlp.get_pipe("ner").labels:
    print(x, ' = ', spacy.explain(x))

CARDINAL  =  Numerals that do not fall under another type
DATE  =  Absolute or relative dates or periods
EVENT  =  Named hurricanes, battles, wars, sports events, etc.
FAC  =  Buildings, airports, highways, bridges, etc.
GPE  =  Countries, cities, states
LANGUAGE  =  Any named language
LAW  =  Named documents made into laws.
LOC  =  Non-GPE locations, mountain ranges, bodies of water
MONEY  =  Monetary values, including unit
NORP  =  Nationalities or religious or political groups
ORDINAL  =  "first", "second", etc.
ORG  =  Companies, agencies, institutions, etc.
PERCENT  =  Percentage, including "%"
PERSON  =  People, including fictional
PRODUCT  =  Objects, vehicles, foods, etc. (not services)
QUANTITY  =  Measurements, as of weight or distance
TIME  =  Times smaller than a day
WORK_OF_ART  =  Titles of books, songs, etc.


In [111]:
def findBestCombo():
    rest = ['CARDINAL', 'DATE', 'EVENT', 'LAW', 'MONEY', 'ORDINAL', 'PERCENT', 'QUANTITY', 'TIME']
    base = ['FAC','GPE','LANGUAGE','LOC','NORP','ORG','PERSON','PRODUCT','WORK_OF_ART']

    combo = []

    for i in range(len(rest)):
        combo += [list(x) for x in combinations(rest, i)]

    best = []
    bestF1 = 0
    bestRe = 0
    bestPre = 0
    bestAcc = 0

    for c in combo:
        l = base + c
        tp, tn, fp, fn, _ = calcPreformance(doc, labels, l)
        recall, precicion, accuracy, f1 = showResults(tp, tn, fp, fn, False)

        if f1 > bestF1:
            bestF1 = f1
            best = l
            bestRe = recall
            bestPre = precicion
            bestAcc = accuracy
    
    print('Best combo = ', best)
    print('f1 = ', bestF1)
    print('recall = ', bestRe)
    print('precision = ', bestPre)
    print('accuracy = ', bestAcc)
    
findBestCombo()



Best combo =  ['FAC', 'GPE', 'LANGUAGE', 'LOC', 'NORP', 'ORG', 'PERSON', 'PRODUCT', 'WORK_OF_ART', 'EVENT', 'LAW']
f1 =  0.5523689990586759
recall =  0.40569690265486724
precision =  0.8651464517397287
accuracy =  0.8373140542791173


In [123]:
def mistakes(doc, labels):
    l = ['FAC', 'GPE', 'LANGUAGE', 'LOC', 'NORP', 'ORG', 'PERSON', 'PRODUCT', 'WORK_OF_ART', 'EVENT', 'LAW']
    _, _, _, _, falseNegativesIndexes = calcPreformance(doc, labels, l)
    
    fnLabels = []
    curr = 0
    for i in range(len(labels)):
        try:
            if i == falseNegativesIndexes[curr]:
                fnLabels.append((labels[i][0], 'fn'))
                curr += 1
            else:
                fnLabels.append((labels[i][0], 'O'))
        except:
            print(curr)
    showLabels(fnLabels)
mistakes(doc, labels)
    

6447
6447
6447
0          1   2  3  4      5 6      7  8        9         10   11         13   14        16  17     
-DOCSTART- Dat is in Italië , Spanje of Engeland misschien geen probleem , maar volgens ' Der Kaiser 

  19 20        21    
' in Duitsland wel . 

23     
sectie 

24     
Zürich 

25     
editie 

26   28       30      32      33 34        35       36          37 38  39        40 41          43     
De ' kleine ' clubs , waarvan de begroting dikwijls afhankelijk is van opleiding en transfers , klagen 

44    45 46   47   48  49          
steen en been over dat voornemen . 

51     52   53   54  55 56         57     58       59 60  61          
Andere zijn bang dat de salarissen straks helemaal de pan uitrijzen . 

63     
auteur 

64   65 66   67   68     69 70          71  72 73                 74 75         76  77  78    79    
Uefa en Fifa zijn bereid de afschaffing van de transfervergoeding te aanvaarden als dat enkel geldt 

80   81          82    83  84 85     
v

   912    
De Morgen 

913             
publicatiedatum 

914    
sectie 

915      916   917    919            922        923  924      925   926 927 928        930    932    
Achteraf bleek dat de machines en de onderdelen geen eigendom waren van het bedrijf en dat ze gewoon 

933    934      935  936 937         938        
werden vervoerd naar hun rechtmatige eigenaar . 

940        941                
-DOCSTART- Algemeen ( 1 & 3 ) 

  
1 

948  949 950                  951  952    953 954    955    957            958              959      
Heel wat conjunctuuranalisten zien daarin een bewijs dat de renteverhoging contraproductief werkt en 

   962         963      965    966        967       969  970       
de economische groei in Europa voortijdig dreigt te doen stikken . 

   973      974      975  976       977      978  979      980         981 982 983   984     985 986 
De Europese Centrale Bank verhoogde gisteren haar centraal rentetarief met een kwart procent tot 4,5 

987 

dotcommers en werden anderen van de cash-burn-dood gered via een overname . 

1936 1937       1939 1940            1943               1946          1949       
Toch blijft het zeer moeilijk om het potentieel van elk bedrijf in te schatten . 

1951        1954   
Koen Van de Steene 

    1956     1957        1959 1960           1963               1966    1967    1968             1971   
Het maximale verlies van 23,5 procent van de Nasdaq-index is de laatste maanden omgebogen in een lichte 

1972          1976    1977     1979        
winst ( + 2.5 procent t.o.v. 1 januari ) . 

1982   
auteur 

1983   
sectie 

   1985   
De Morgen 

1986     1987 1988      1990        1991      1992           1994    
Winnaars zijn alvast de Amerikaanse bedrijven CommerceOne en Ariba . 

1996         1998        2000    2001          2004      
Gisteren nog knock-out , vandaag opnieuw in de running . 

2006     
Algemeen 

   2008       2010           2013      2014        2017      2018 2019 2020    


De belastingverlaging die minister van Financiën Didier Reynders ( PRL ) voorstelt , kan slechts deels 

      2892              2895 2896      
op de instemming van Di Rupo rekenen . 

2898       2900        2902       2904       2906 2907           2909  2910      2911     
Trouwens , niemand mag tijdens de komende -- zeer belangrijke -- weken benadeeld worden . 

         2916     2917      2918 2919         2922        2923     
" Ook de volgende generatie moet delen in het beschikbare geld . " 

2926        2929      2930 
vrijdag , 1 september 2000 

2931   2932            2934        2935          
Zondag congresseren de Franstalige socialisten . 

2937    
Brussel 

  2939     2941 2942     2944            2947 2948         2949         
" Vroeg of laat moet een regering wel de hele problematiek bekijken . " 

2952     2954      2956           2960 2961 2962 2963         2966                2970             2973     
Voor hem staat het vast dat er al snel geld moet komen in het

   3970   3971   3972     3973           3976        3978 3979    3980       3981       3984     3986 
Na afloop maakte Gabriëls bekend dat hij vandaag met zijn Vlaamse ambtgenoot Vera Dua ( Agalev ) gaat 

3987         3990                 3993          3995                      3997        
praten om de mogelijkheid tot het uitreiken van overbruggingskredieten te bespreken . 

        4001       4003     4004    4005 4006     
Dat zou echter ten vroegste volgend jaar kunnen . 

   4009   
De Morgen 

4010            
publicatiedatum 

4011     4012         4014     4015    4016          4020         4022                    4026          
Federaal minister van Economie Charles Picqué ( PS ) kondigde een zespuntenplan aan om de huisbrandolie 

4027    4029    4030     4031          4033     
voor de laagste inkomens betaalbaar te houden . 

4035        4038      4039 
vrijdag , 1 september 2000 

4040        4042        4044     4045         4047      4048   4049             4053    40

...

   63153  63154    63155            63157         63160     63162                               63171    
Ze kunnen jongeren bijvoorbeeld ook leren hoe een budget te beheren , of hoe met hun tijd om te springen 

  
. 

63173      
Immoreel ? 

        63178 63179                  63184              
" Nee , gsm's hoeven niet per se een probleem te zijn . 

     63189    63190 63191          63194             63198  63199    63200         63201   63202       
Maar jongeren leven sowieso in een cultuur dat ze op zoveel mogelijk verschillende plekken willen zijn 

  
. 

        63207    63209                                       63218             63221  63222         
Het kan nooit de bedoeling zijn dat een kind bang is dat het permanent door de ouders gecontroleerd 

    63224    
kan worden . 

63226         63229     63230    63232                63235  63236       63237                      
Vooral in het secundair neemt de populariteit van het gadget waanzinnige proporties aan , e

Laten wij als land de basis leggen voor een soort Marshall-plan voor aids en ontwikkeling , als wij 

64510  64511         64513      64515  64516    64517  64518             64521   
willen voorkomen dat anderen de eerste medische oorlog ontketenen : Aids Storm . 

64523       64526                 64528                     64529         
Laten we de 0,7-procentsnorm voor ontwikkelingssamenwerking respecteren . 

               64534    64536      64538              64542         64544            64548 64549    
Hier komt mijn kreet en pleidooi : laten wij als rijk Vlaanderen en België , met een grote ervaring 

   64551         64552        64555        64557               64560                   64563    64564 
in sub-Saharisch Afrika , mee instappen in partenariaten en op internationaal vlak het voortouw nemen 

  
. 

    64567         64570               64574      64576    64578                64582     64584        
Een jongen die nu vijftien jaar is in Botswana , heeft 90 proc

65815       65817       65818                65823      65825    65826       65827 65828             
Dankzij het Nederlandse Hessing , dat ook in België een heleboel Amerikaanse Fords verdeelt , is hij 

     65833      65835           65838          
hier sinds kort echter wel weer verkrijgbaar . 

65840       65841       65843  
Amerikaanse topper naar België 

   65845  
De Morgen 

    65847      65849      65850              65853 65854   65855        65858        65859           
Met onder meer elektrisch verstelbare en met leder beklede zetels , een automatische airconditioning 

      65862                    65865                 65867              65870           65872     
, een snelheidsregelaar en een muziekinstallatie met cd-wisselaar is de uitrusting heel luxueus . 

           65877                 65880       65882         65884                    65890     65891      
Zo zou hij bijvoorbeeld heel wat steviger en veiliger zijn geworden , wat hem in de officiële crasht

op de expositie te zien , EB ) . 

   67152  
De Morgen 

          67156        67159        
" Ook het licht is erg belangrijk . 

67161       67163 67164             67167     67168  67169 
Beelden van Johan Parmentier bij de Stichting George Grard 

                 67175   67176                    67181           67185       67186 67187        
' De taal die de materie spreekt , moet mijn taal worden " , zegt beeldhouwer Johan Parmentier . 

67189    67191            67193                 67196                  67201     67203        67205    
Vormen , contourlijnen en kleurgradaties die me treffen leg ik vast in potlood , houtskool en olieverf 

  
. 

             67210                  67215               67219        
Het moet een steen zijn die je kunt beleven door hem te bekijken . " 

67222   
Cultuur 

  67224                       67229                     67235            67239             67242     
" Eigenlijk hoef ik zelf niet zoveel te doen , maar elk beeld is wel he

Die motor kwam de voorbije maanden al terecht in de Laguna en de ' gewone ' Scénic en is nu ook verkrijgbaar 

                 68509  68510                  68514        68515      68516        
in de op de vier wielen aangedreven en als een terreinwagen uitgedoste Scénic RX4 . 

68519   68520                        68527              68531    68533          68536  68537       68539    
Renault rekent er dan ook op dat die diesel snel 65 tot zelfs 70 procent van de totale verkoop zal uitmaken 

  
. 

68541  
auteur 

                        68548   68549   68551                   68557                         68564  
Met de dCi kost de auto 950.000 frank ( 23.550 euro ) , wat net evenveel is als met de 140 pk sterke 

68565                 
2-literbenzinemotor . 

68567       68570          
vrijdag , 1 september 2000 

68572      
publicatie 

   68574       68575       68576 68577           68580 68581           68584       68585        68587       
De volledigere verbranding zorgt

In [112]:
t = list(nlp.get_pipe("ner").labels)

for i in range(len(t)):
    combo += [list(x) for x in combinations(t, i)]
    
len(combo)

262654